In [1]:
import argparse
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import numpy as np
from torch.autograd import Variable


import os
os.chdir('../')

from experiment_abstract.dataset import Dictionary, VQAFeatureDataset
exp = './experiment_abstract/'
from models import my_model as base_model

In [2]:
# https://github.com/szagoruyko/pytorchviz/blob/master/examples.ipynb
# make_dot(model(x), params=dict(model.named_parameters()))

In [3]:
dictionary = Dictionary.load_from_file(exp+'data/dictionary.pkl')
train_dset = VQAFeatureDataset('train', dictionary, dataroot=exp+'data/')
eval_dset = VQAFeatureDataset('val', dictionary, dataroot=exp+'data/')
batch_size = 256
num_hid = 1024

constructor = 'build_baseline0_newatt'
model = getattr(base_model, constructor)(train_dset, num_hid).cuda()
model.w_emb.init_embedding(exp+'data/glove6b_init_300d.npy')

train_loader = DataLoader(train_dset, batch_size, shuffle=True, num_workers=1)
eval_loader =  DataLoader(eval_dset, batch_size, shuffle=True, num_workers=1)

model = nn.DataParallel(model).cuda()

loading dictionary from ./experiment_abstract/data/dictionary.pkl
loading features from h5 file
answer_path ./experiment_abstract/data/cache/train_target.pkl
loading features from h5 file
answer_path ./experiment_abstract/data/cache/val_target.pkl


In [4]:
model_path = exp+'saved_models/my_model_2/model.pth'
model_params = torch.load(model_path)
model.load_state_dict(model_params, strict=False)
model.train(False)
model.eval() 
pass

In [5]:
filtered = {}
dict_params = dict(model_params)
for key in dict_params:
    if 'classifier' in key:
        filtered[key] = dict_params[key]

Weight normalization is a reparameterization that decouples the magnitude
of a weight tensor from its direction. This replaces the parameter specified
by name (e.g. "weight") with two parameters: one specifying the magnitude
(e.g. "weight_g") and one specifying the direction (e.g. "weight_v").

In [6]:
#[1024 top-down, 1024 bottom-up]
filtered['module.classifier.main.0.weight_v'].size()

torch.Size([2048, 2048])

In [7]:
filtered['module.classifier.main.3.weight_v'].size()

torch.Size([426, 2048])

In [8]:
sorted(filtered.keys())

['module.classifier.main.0.bias',
 'module.classifier.main.0.weight_g',
 'module.classifier.main.0.weight_v',
 'module.classifier.main.3.bias',
 'module.classifier.main.3.weight_g',
 'module.classifier.main.3.weight_v']

In [9]:
# check for question type the distribution over the values on the Relu, count 1 if > 0.

In [10]:
def train(model, train_loader, eval_loader):
    preds = []
    for i, (v, b, q, a) in enumerate(train_loader):
        v = Variable(v).cuda()
        b = Variable(b).cuda()
        q = Variable(q).cuda()
        a = Variable(a).cuda()
        pred = model(v, b, q, a)
        preds.append(pred)
    return np.array(preds)

In [11]:
responses = train(model, train_loader, eval_loader)

RuntimeError: CUDA error: out of memory